In [83]:
import pandas as pd
import time

import tensorflow as tf
from gensim.models import Word2Vec


def get_sess(batch_size):
    tf.reset_default_graph()
    graph = tf.get_default_graph()
    lt = tf.Variable(
        pd.read_pickle('./data/lookup_table'),
        dtype=tf.float32
    )
    sen = tf.truncated_normal([batch_size, 64, 512])
    batch = tf.map_fn(lambda word: tf.map_fn(
        lambda x: tf.cast(tf.argmin(
            tf.reduce_sum(tf.abs((lt-x)), axis=0)
        ), dtype=tf.float32), word
    ), sen)
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    return batch, sess, graph


def batch2sens(batch):
    
    i2w = Word2Vec.load('./data/w2v_model').wv.index2word
    
    def get_sen(l):
        ws = [i2w[i] for i in l]
        sen = ' '.join(w.split('-')[0] for w in ws)
        return sen

    batch = batch.astype(int)
    se_sen = pd.Series([get_sen(l) for l in batch])
    
    return se_sen

In [84]:
batch, sess, graph = get_sess(2)
s = time.time()
se = batch2sens(sess.run(batch))
print(time.time()-s)

10.593078136444092


In [85]:
se

0    을까 어떤 함 큰 게 인데 인데 세상 화 재미 함 함 처음 재미 알 남자 두 함 화...
1    함 함 큰 알 재미 평범 알 대사 해 건지 알 함 함 두 어떤 큰 함 처음 재미 함...
dtype: object

In [106]:
graph.get_all_collection_keys()

['trainable_variables', 'variables', 'while_context', '']